In [139]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [140]:
# cite from： https://blog.csdn.net/weixin_42029733/article/details/89922575
from itertools import combinations

def combine(temp_list, n):
    '''根据n获得列表中的所有可能组合（n个元素为一组）'''
    temp_list2 = []
    for c in combinations(temp_list, n):
        temp_list2.append(c)
    return temp_list2

# an example
list1 = ['a', 'b', 'c']
end_list = []
for i in range(len(list1)+1):
    end_list.extend(combine(list1, i))
    # print(type(combine(list1, i)))
print(end_list)

[(), ('a',), ('b',), ('c',), ('a', 'b'), ('a', 'c'), ('b', 'c'), ('a', 'b', 'c')]


In [141]:
def compute_SHAP(topic_a, cur_arg):

    model = grad.semantics.QuadraticEnergyModel()
    model.approximator = grad.algorithms.RK4(model)
    model.BAG = grad.BAG("../../bags/aaa_test.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    arg_name_all = []

    for arg in model.BAG.arguments.values():
        arg_name_all.append(arg.name)
        # print((arg.name,arg.strength))
    arg_name_no_topic_arg = arg_name_all.copy()
    arg_name_no_topic_arg.remove(topic_a)

    print(arg_name_all)
    print(arg_name_no_topic_arg)

    # remove current argument
    arg_name_no_topic_no_cur = arg_name_no_topic_arg.copy()
    arg_name_no_topic_no_cur.remove(cur_arg)
    print(arg_name_no_topic_no_cur)
    print("----------------------------")

    # compute all the combinations
    all_possible_combinations = []
    for i in range(len(arg_name_no_topic_no_cur)+1):
        all_possible_combinations.extend(combine(arg_name_no_topic_no_cur, i))
    print(f"all_possible_combinations:{all_possible_combinations}")

    # store the initial weights
    arg_initial_weight = []
    # arg_initial_weight_dict = {}
    for arg in model.BAG.arguments.values():
        arg_initial_weight.append((arg.name,arg.initial_weight))

    print("All the base score and strength in QBAF:")
    for arg in model.BAG.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))

    coefficient = []
    difference = []

    n = len(arg_name_all)-1 # 1 is topic argument
    for i in all_possible_combinations:
        print("----------------------------")
        print(f"Combination:{i}")

        # remove arg(j) from the current combination i
        for j in i: # first remove the argument from the combination
            model.BAG.remove_argument(j)

        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

        # print 1st: before remove cur arg
        print("Initial and final strength (1st):")
        for arg in model.BAG.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = model.BAG.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength
        print(f"strength_before_drop_cur:{strength_before_drop_cur}")

        # remove cur arg
        model.BAG.remove_argument(cur_arg)
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

        # print 2nd: after remove cur arg
        print("Initial and final strength (2nd):")
        for arg in model.BAG.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = model.BAG.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength
        print(f"strength_after_drop_cur:{strength_after_drop_cur}")

        # compute the diff of topic arg before and after removing
        difference.append(strength_before_drop_cur-strength_after_drop_cur)
        print(f"diff={strength_before_drop_cur-strength_after_drop_cur}")

        # compute the coefficient for SHAP value
        S = len(i)
        coefficient.append(math.factorial(n-1-S)*math.factorial(S)/math.factorial(n))
        model.BAG = grad.BAG("../../bags/aaa_test.bag")
        model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    print("----------------------------")
    print("Conclusion:")
    print(coefficient)
    print(difference)
    attribution = sum(np.multiply(coefficient, difference))
    print(attribution)

    # initializing: reset to original initial weight
    model = grad.semantics.QuadraticEnergyModel()
    model.approximator = grad.algorithms.RK4(model)
    model.BAG = grad.BAG("../../bags/aaa_test.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

    return attribution

In [142]:
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG("../../bags/aaa_test.bag")
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 7.499999999999989
Argument(name=A, weight=0.5, strength=0.8762604661060156)
Argument(name=B, weight=0.5, strength=0.8762604661060156)
Argument(name=C, weight=0.5, strength=0.8762604661060156)



0.0009671618328896773

In [143]:
# compute attribution for one arg
topic_a = "A"
all_args = model.BAG.get_arguments()
all_args_names = []
for i in all_args:
    all_args_names.append(i.get_name())
print(all_args_names)

all_args_names_without_topic = all_args_names.copy()
all_args_names_without_topic.remove(topic_a)
print(all_args_names_without_topic)

attribution = {}
for cur_arg in all_args_names_without_topic:
    attribution[cur_arg]=compute_SHAP(topic_a, cur_arg)
print(f"attribution: \n{attribution}")

['A', 'B', 'C']
['B', 'C']
QuadraticEnergyModel, RK4
Time: 7.499999999999989
Argument(name=A, weight=0.5, strength=0.8762604661060156)
Argument(name=B, weight=0.5, strength=0.8762604661060156)
Argument(name=C, weight=0.5, strength=0.8762604661060156)

['A', 'B', 'C']
['B', 'C']
['C']
----------------------------
all_possible_combinations:[(), ('C',)]
All the base score and strength in QBAF:
('A', 0.5, 0.8762604661060156)
('B', 0.5, 0.8762604661060156)
('C', 0.5, 0.8762604661060156)
----------------------------
Combination:()
QuadraticEnergyModel, RK4
Time: 0.1
Argument(name=A, weight=0.5, strength=0.8763498487445333)
Argument(name=B, weight=0.5, strength=0.8763498487445333)
Argument(name=C, weight=0.5, strength=0.8763498487445333)

Initial and final strength (1st):
('A', 0.5, 0.8763498487445333)
('B', 0.5, 0.8763498487445333)
('C', 0.5, 0.8763498487445333)
strength_before_drop_cur:0.8763498487445333
QuadraticEnergyModel, RK4
Time: 7.499999999999989
Argument(name=A, weight=0.5, strength

In [144]:
print(f"base score of the topic_arg: {model.BAG.arguments[topic_a].initial_weight}")
# sum of all attributions
total_sum = sum(attribution.values())
print(f"sum of all attributions: {total_sum}")
print(f"sum of all attributions + BS: {total_sum+model.BAG.arguments[topic_a].initial_weight}")
print(f"strength of the topic_arg: {model.BAG.arguments[topic_a].strength}")

base score of the topic_arg: 0.5
sum of all attributions: 0.3754398206856041
sum of all attributions + BS: 0.8754398206856041
strength of the topic_arg: 0.8762604661060156
